In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocessed-train/final_preprocessed.csv


In [9]:
path_link="/kaggle/input/preprocessed-train/final_preprocessed.csv"
df=pd.read_csv(path_link)
df=df.sample(100000)

In [10]:
X=df['review_text']
y=df['rating']

In [11]:
df['review_text'].fillna('', inplace=True)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

## Logistic Regression

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
# Step 4: Create a logistic regression model
logreg = LogisticRegression(max_iter=1000)

# Step 5: Fit the model to the oversampled training data
logreg.fit(X_train_tfidf, y_train)

# Step 6: Make predictions on the test set
y_pred = logreg.predict(X_test_tfidf)
# Calculate F1 Score
f1_logreg = f1_score(y_test,y_pred, average='weighted')
print("Logistic Regression F1 Score: {:.2f}".format(f1_logreg))

Logistic Regression F1 Score: 0.46


## K means

In [17]:
from sklearn.cluster import KMeans

number_of_clusters = 6
# Create the classifier
km = KMeans(n_clusters=number_of_clusters)
# Put them into categories
km.fit(X_train_tfidf)
test_clusters = km.predict(X_test_tfidf)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [18]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test, test_clusters, average='micro')

print(f"F1 Score: {f1}")

F1 Score: 0.19666666666666666


## Multinomail NB


In [19]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
nb_classifier = MultinomialNB()

# Train the classifier on the training data
nb_classifier.fit(X_train_tfidf, y_train)

# Predict the labels for both training and testing data
test_predictions = nb_classifier.predict(X_test_tfidf)

# Evaluate the F1 score for the testing data
f1 = f1_score(y_test, test_predictions, average='micro')
print(f"F1 Score: {f1}")

F1 Score: 0.3738333333333333


## SVM

In [20]:
from sklearn import svm
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf, y_train)

predictions_SVM = SVM.predict(X_test_tfidf)
f1 = f1_score(y_test, predictions_SVM, average='weighted')
print("F1 Score: {:.2f}".format(f1))

F1 Score: 0.48


## Random Forest

In [22]:
from sklearn.ensemble import RandomForestClassifier 
# Initialize Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model
rf_model.fit(X_train_tfidf, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test_tfidf)

# Evaluate the model
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"F1 Score: {f1}")

F1 Score: 0.38415554976086713
